### Create web analytics dataset

In [2]:
import pandas as pd
import numpy as np
from faker import Faker
import random
import datetime

In [6]:
# Initialize Faker
fake = Faker()

# Set random seed for reproducibility
random.seed(42)
fake.seed_instance(42)

# Parameters
num_days = 90  # Approximately 3 months
total_records = num_days  # One record per day

# Define traffic sources and device types with some inconsistencies
traffic_sources = ['Organic', 'Direct', 'Referral', 'Social', 'Email', 'Paid Search', 'Display Ads', 'Social Media', 'Organic Search']
device_types = ['Desktop', 'Mobile', 'Tablet', 'desktop', 'mobile', 'tab']
countries = ['United States', 'Canada', 'United Kingdom', 'Australia', 'Germany', 'France', 'India', 'Brazil', 'Japan', 'China']

# Define a list of platforms from the advertising data to align traffic sources
ad_platforms = ['Google Ads', 'Facebook Ads', 'Instagram Ads', 'LinkedIn Ads', 'GAds', 'FB Ads', 'Insta Ads']

# Generate a list of dates for the past 3 months
date_range = pd.date_range(end=pd.Timestamp.today(), periods=num_days).tolist()

# Initialize list to hold all website analytics records
website_analytics = []

for single_date in date_range:
    # Introduce inconsistent date formats (10% of the time)
    if random.random() > 0.1:
        date_str = single_date.strftime('%Y-%m-%d')
    else:
        date_str = single_date.strftime('%d/%m/%Y')
    
    # Introduce missing values (5% for 'traffic_source' and 'device_type')
    traffic_source = random.choice(traffic_sources) if random.random() > 0.05 else None
    device_type = random.choice(device_types) if random.random() > 0.05 else None
    
    # Introduce erroneous entries (2% negative values for numeric metrics)
    page_views = random.randint(1000, 100000) if random.random() > 0.02 else -random.randint(1000, 100000)
    unique_visitors = random.randint(500, 50000) if random.random() > 0.02 else -random.randint(500, 50000)
    sessions = random.randint(800, 80000) if random.random() > 0.02 else -random.randint(800, 80000)
    bounce_rate = round(random.uniform(20, 80), 2) if random.random() > 0.02 else -round(random.uniform(20, 80), 2)
    average_session_duration = random.randint(30, 600) if random.random() > 0.02 else -random.randint(30, 600)
    conversions = random.randint(10, 1000) if random.random() > 0.02 else -random.randint(10, 1000)
    
    # Assign traffic source from advertising platforms with some probability
    # This introduces a relation between ad platforms and traffic sources
    if traffic_source in ['Social Media', 'Paid Search', 'Display Ads']:
        traffic_source = random.choice(ad_platforms)
    
    # Assign country
    country = random.choice(countries)
    
    # Assign device type with consistency
    device_type = device_type.capitalize() if device_type else device_type
    
    # Create the website analytics record
    analytics_record = {
        'date': date_str,
        'page_views': page_views,
        'unique_visitors': unique_visitors,
        'sessions': sessions,
        'bounce_rate': bounce_rate,
        'average_session_duration': average_session_duration,
        'traffic_source': traffic_source,
        'device_type': device_type,
        'country': country,
        'conversions': conversions
    }
    
    website_analytics.append(analytics_record)

# Convert to DataFrame
website_analytics_df = pd.DataFrame(website_analytics)

# Introduce Duplicates (2% of the data)
duplicates = website_analytics_df.sample(frac=0.02, random_state=42)
website_analytics_df = pd.concat([website_analytics_df, duplicates], ignore_index=True)

# Introduce Additional Outliers by adding some extreme values (20 records)
outlier_records = []
for _ in range(20):
    outlier_date = random.choice(date_range).strftime('%Y-%m-%d')
    outlier_record = {
        'date': outlier_date,
        'page_views': random.randint(1000000, 10000000),  # Extremely high page views
        'unique_visitors': random.randint(500000, 5000000),  # Extremely high unique visitors
        'sessions': random.randint(800000, 8000000),  # Extremely high sessions
        'bounce_rate': round(random.uniform(5, 95), 2),  # Extreme bounce rates
        'average_session_duration': random.randint(10, 3600),  # Extremely long session durations
        'traffic_source': random.choice(ad_platforms),
        'device_type': random.choice(['Desktop', 'Mobile', 'Tablet']),
        'country': random.choice(countries),
        'conversions': random.randint(1000, 10000)  # Extremely high conversions
    }
    outlier_records.append(outlier_record)

# Convert outliers to DataFrame
outliers_df = pd.DataFrame(outlier_records)

# Concatenate outliers with the main DataFrame
website_analytics_df = pd.concat([website_analytics_df, outliers_df], ignore_index=True)

# Shuffle the DataFrame to mix outliers and duplicates
website_analytics_df = website_analytics_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Parse 'date' column to datetime for sorting
website_analytics_df['parsed_date'] = pd.to_datetime(website_analytics_df['date'], dayfirst=True, errors='coerce')

# Sort by 'parsed_date'
website_analytics_df = website_analytics_df.sort_values(by='parsed_date').drop('parsed_date', axis=1).reset_index(drop=True)

# Save to CSV
website_analytics_df.to_csv('website_analytics_dirty.csv', index=False)
print("Dirty Website Analytics Data Generated:")
print(website_analytics_df.head())


Dirty Website Analytics Data Generated:
         date  page_views  unique_visitors  sessions  bounce_rate  \
0  2024-07-21       14434            36241     56102        25.62   
1  2024-07-22       21926            22798     29021        40.20   
2  2024-07-23       50615            19713     48200        62.27   
3  2024-07-24       47566            17996     10158        30.27   
4  2024-07-25       53581            14326     75141        38.88   

   average_session_duration traffic_source device_type    country  conversions  
0                        57           None      Mobile      Japan          743  
1                       129          Email     Desktop      China          362  
2                       263           None     Desktop  Australia           91  
3                       197         FB Ads      Tablet      Japan          286  
4                       435        Organic     Desktop     Brazil          668  


/tmp/ipykernel_86239/2425329212.py:107: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  website_analytics_df['parsed_date'] = pd.to_datetime(website_analytics_df['date'], dayfirst=True, errors='coerce')


### Create CRM dataset

In [9]:
# Set random seed for reproducibility
random.seed(42)
fake.seed_instance(42)

# Parameters
num_customers = 1000  # Total number of customers
total_records = num_customers

# Define customer segments, loyalty program status, and preferred channels
customer_segments = ['Bronze', 'Silver', 'Gold', 'Platinum']
loyalty_program_status = ['Yes', 'No']
preferred_channels = ['Email', 'Phone', 'SMS', 'None']

# Define countries (consistent with other datasets)
countries = ['United States', 'Canada', 'United Kingdom', 'Australia', 'Germany', 'France',
             'India', 'Brazil', 'Japan', 'China']

# Generate a list of dates for the past 3 years (assuming CRM data spans longer)
date_range = pd.date_range(end=pd.Timestamp.today(), periods=1095).tolist()  # Approximately 3 years

# Initialize list to hold all CRM records
crm_records = []

for customer_num in range(1, num_customers + 1):
    customer_id = f'cust_{customer_num}'
    first_name = fake.first_name()
    last_name = fake.last_name()
    email = fake.email()
    phone_number = fake.phone_number()
    address = fake.street_address()
    city = fake.city()
    state = fake.state()
    country = random.choice(countries)
    zip_code = fake.zipcode()
    
    # Generate signup date within the past 3 years
    signup_date = fake.date_between(start_date='-3y', end_date='today')
    
    # Generate last purchase date after signup_date
    last_purchase_date = fake.date_between(start_date=signup_date, end_date='today')
    
    # Total purchases between 1 and 100
    total_purchases = random.randint(1, 100)
    
    # Total spent between $10 and $10,000
    total_spent = round(random.uniform(10, 10000), 2)
    
    # Loyalty program member
    loyalty_member = random.choice(loyalty_program_status)
    
    # Customer segment based on total_spent
    if total_spent < 100:
        segment = 'Bronze'
    elif 100 <= total_spent < 500:
        segment = 'Silver'
    elif 500 <= total_spent < 2000:
        segment = 'Gold'
    else:
        segment = 'Platinum'
    
    # Preferred communication channel
    preferred_channel = random.choice(preferred_channels)
    
    # Feedback score between 1 and 5
    feedback_score = random.randint(1, 5)
    
    # Support tickets opened between 0 and 20
    support_tickets = random.randint(0, 20)
    
    # Churn risk based on last_purchase_date
    days_since_last_purchase = (pd.Timestamp.today() - pd.Timestamp(last_purchase_date)).days
    if days_since_last_purchase < 30:
        churn_risk = 'Low'
    elif 30 <= days_since_last_purchase < 90:
        churn_risk = 'Medium'
    else:
        churn_risk = 'High'
    
    # Introduce missing values (5% for 'phone_number', 'preferred_channel')
    phone_number = phone_number if random.random() > 0.05 else None
    preferred_channel = preferred_channel if random.random() > 0.05 else None
    
    # Introduce erroneous entries (2% negative values for 'total_purchases', 'total_spent')
    total_purchases = total_purchases if random.random() > 0.02 else -total_purchases
    total_spent = total_spent if random.random() > 0.02 else -total_spent
    
    # Create the CRM record
    crm_record = {
        'customer_id': customer_id,
        'first_name': first_name,
        'last_name': last_name,
        'email': email,
        'phone_number': phone_number,
        'address': address,
        'city': city,
        'state': state,
        'country': country,
        'zip_code': zip_code,
        'signup_date': signup_date.strftime('%Y-%m-%d'),
        'last_purchase_date': last_purchase_date.strftime('%Y-%m-%d'),
        'total_purchases': total_purchases,
        'total_spent': total_spent,
        'loyalty_program_member': loyalty_member,
        'customer_segment': segment,
        'preferred_channel': preferred_channel,
        'feedback_score': feedback_score,
        'support_tickets_opened': support_tickets,
        'churn_risk': churn_risk
    }
    
    crm_records.append(crm_record)

# Convert to DataFrame
crm_df = pd.DataFrame(crm_records)

# Introduce Duplicates (2% of the data)
duplicates = crm_df.sample(frac=0.02, random_state=42)
crm_df = pd.concat([crm_df, duplicates], ignore_index=True)

# Introduce Additional Outliers by adding some extreme values (20 records)
outlier_records = []
for _ in range(20):
    outlier_customer_num = random.randint(1, num_customers)
    outlier_customer_id = f'cust_{outlier_customer_num}'
    outlier_first_name = fake.first_name()
    outlier_last_name = fake.last_name()
    outlier_email = fake.email()
    outlier_phone_number = fake.phone_number()
    outlier_address = fake.street_address()
    outlier_city = fake.city()
    outlier_state = fake.state()
    outlier_country = random.choice(countries)
    outlier_zip_code = fake.zipcode()
    
    # Extreme signup and last purchase dates
    outlier_signup_date = fake.date_between(start_date='-5y', end_date='-3y')
    outlier_last_purchase_date = fake.date_between(start_date=outlier_signup_date, end_date='-3y')
    
    # Extremely high total purchases and total spent
    outlier_total_purchases = random.randint(500, 1000)
    outlier_total_spent = round(random.uniform(20000, 100000), 2)
    
    # Loyalty program member
    outlier_loyalty_member = 'Yes'
    
    # Customer segment
    outlier_segment = 'Platinum'
    
    # Preferred communication channel
    outlier_preferred_channel = 'Email'
    
    # Feedback score
    outlier_feedback_score = 5
    
    # Support tickets opened
    outlier_support_tickets = random.randint(5, 50)
    
    # Churn risk
    outlier_churn_risk = 'High'
    
    # Create the outlier CRM record
    outlier_record = {
        'customer_id': outlier_customer_id,
        'first_name': outlier_first_name,
        'last_name': outlier_last_name,
        'email': outlier_email,
        'phone_number': outlier_phone_number,
        'address': outlier_address,
        'city': outlier_city,
        'state': outlier_state,
        'country': outlier_country,
        'zip_code': outlier_zip_code,
        'signup_date': outlier_signup_date.strftime('%Y-%m-%d'),
        'last_purchase_date': outlier_last_purchase_date.strftime('%Y-%m-%d'),
        'total_purchases': outlier_total_purchases,
        'total_spent': outlier_total_spent,
        'loyalty_program_member': outlier_loyalty_member,
        'customer_segment': outlier_segment,
        'preferred_channel': outlier_preferred_channel,
        'feedback_score': outlier_feedback_score,
        'support_tickets_opened': outlier_support_tickets,
        'churn_risk': outlier_churn_risk
    }
    
    outlier_records.append(outlier_record)

# Convert outliers to DataFrame
outliers_df = pd.DataFrame(outlier_records)

# Concatenate outliers with the main DataFrame
crm_df = pd.concat([crm_df, outliers_df], ignore_index=True)

# Shuffle the DataFrame to mix outliers and duplicates
crm_df = crm_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Parse 'signup_date' column to datetime for sorting
crm_df['parsed_signup_date'] = pd.to_datetime(crm_df['signup_date'], dayfirst=True, errors='coerce')

# Sort by 'parsed_signup_date'
crm_df = crm_df.sort_values(by='parsed_signup_date').drop('parsed_signup_date', axis=1).reset_index(drop=True)

# Save to CSV
crm_df.to_csv('crm_dirty.csv', index=False)
print("Dirty CRM Data Generated:")
print(crm_df.head())

Dirty CRM Data Generated:
  customer_id first_name last_name                        email  \
0    cust_185    Katelyn     Nunez         justin07@example.net   
1     cust_22     Angela    Garcia       lauradixon@example.org   
2    cust_961      Keith    Taylor  cervantessandra@example.net   
3    cust_491     Olivia    Murphy      gentrylarry@example.com   
4    cust_352       Sean    Sparks      isaiahbrown@example.net   

           phone_number                    address                 city  \
0  001-296-277-8889x221  04094 Wolf Cliff Apt. 246       Lake Scottfort   
1    (732)978-6807x0423          89959 Chad Common       Maldonadoshire   
2    (551)681-2759x1700     4325 Shaffer Mountains         Anthonymouth   
3  001-291-753-2180x113         15682 Keith Plains            Sarahtown   
4   +1-597-503-2043x474   2106 Jones Cape Apt. 460  North Cristinaville   

        state         country zip_code signup_date last_purchase_date  \
0        Ohio         Germany    91314  2019-10

/tmp/ipykernel_86239/343986587.py:197: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  crm_df['parsed_signup_date'] = pd.to_datetime(crm_df['signup_date'], dayfirst=True, errors='coerce')


### Email campaigns

In [7]:
# Set random seed for reproducibility
random.seed(42)
fake.seed_instance(42)

# Parameters
num_campaigns = 50  # Total number of email campaigns
records_per_campaign = 20  # Number of records per campaign (simulating multiple sends or variations)
total_records = num_campaigns * records_per_campaign

# Define possible subject lines, platforms, and other categorical variables
subject_lines = [
    "Exclusive Offer Just for You!",
    "Don't Miss Out on Our Summer Sale",
    "Welcome to Our Newsletter",
    "Limited Time Discount Inside",
    "Your Feedback Matters",
    "New Arrivals Available Now",
    "Special Invitation: Join Us Today",
    "Thank You for Being a Loyal Customer",
    "Unlock Your Premium Features",
    "Last Chance to Save Big!"
]

platforms = ['Google Ads', 'Facebook Ads', 'Instagram Ads', 'LinkedIn Ads', 'GAds', 'FB Ads', 'Insta Ads', 'Email Campaign']
countries = ['United States', 'Canada', 'United Kingdom', 'Australia', 'Germany', 'France', 'India', 'Brazil', 'Japan', 'China']

# Generate a list of dates for the past 3 months
date_range = pd.date_range(end=pd.Timestamp.today(), periods=90).tolist()

# Initialize list to hold all email campaign records
email_campaigns = []

for campaign_num in range(1, num_campaigns + 1):
    campaign_id = f'campaign_{campaign_num}'
    campaign_name = f'Campaign {campaign_num} - {fake.word().capitalize()} Promotion'
    
    for _ in range(records_per_campaign):
        # Introduce inconsistent date formats (10% of the time)
        if random.random() > 0.1:
            date_sent = random.choice(date_range).strftime('%Y-%m-%d')
        else:
            date_sent = random.choice(date_range).strftime('%d/%m/%Y')
        
        # Select a subject line
        subject_line = random.choice(subject_lines)
        
        # Introduce missing values (5% for 'subject_line' and 'platform')
        subject_line = subject_line if random.random() > 0.05 else None
        platform = random.choice(platforms) if random.random() > 0.05 else None
        
        # Introduce erroneous entries (2% negative values for numeric metrics)
        recipient_count = random.randint(1000, 100000) if random.random() > 0.02 else -random.randint(1000, 100000)
        delivered_count = random.randint(900, recipient_count) if (recipient_count > 0 and random.random() > 0.02) else -random.randint(900, recipient_count if recipient_count > 0 else 1000)
        opened_count = random.randint(100, delivered_count) if (delivered_count > 0 and random.random() > 0.02) else -random.randint(100, delivered_count if delivered_count > 0 else 1000)
        clicked_count = random.randint(50, opened_count) if (opened_count > 0 and random.random() > 0.02) else -random.randint(50, opened_count if opened_count > 0 else 1000)
        bounced_count = random.randint(0, 1000) if random.random() > 0.02 else -random.randint(0, 1000)
        unsubscribe_count = random.randint(0, 500) if random.random() > 0.02 else -random.randint(0, 500)
        spam_report_count = random.randint(0, 200) if random.random() > 0.02 else -random.randint(0, 200)
        conversions = random.randint(10, 1000) if random.random() > 0.02 else -random.randint(10, 1000)
        
        # Assign country
        country = random.choice(countries)
        
        # Create the email campaign record
        campaign_record = {
            'campaign_id': campaign_id,
            'campaign_name': campaign_name,
            'date_sent': date_sent,
            'subject_line': subject_line,
            'platform': platform,
            'country': country,
            'recipient_count': recipient_count,
            'delivered_count': delivered_count,
            'opened_count': opened_count,
            'clicked_count': clicked_count,
            'bounced_count': bounced_count,
            'unsubscribe_count': unsubscribe_count,
            'spam_report_count': spam_report_count,
            'conversions': conversions
        }
        
        email_campaigns.append(campaign_record)

# Convert to DataFrame
email_campaigns_df = pd.DataFrame(email_campaigns)

# Introduce Duplicates (2% of the data)
duplicates = email_campaigns_df.sample(frac=0.02, random_state=42)
email_campaigns_df = pd.concat([email_campaigns_df, duplicates], ignore_index=True)

# Introduce Additional Outliers by adding some extreme values (10 records)
outlier_records = []
for _ in range(10):
    outlier_campaign_num = random.randint(1, num_campaigns)
    outlier_campaign_id = f'campaign_{outlier_campaign_num}'
    outlier_campaign_name = f'Campaign {outlier_campaign_num} - {fake.word().capitalize()} Blitz'
    outlier_date = random.choice(date_range).strftime('%Y-%m-%d')
    
    outlier_record = {
        'campaign_id': outlier_campaign_id,
        'campaign_name': outlier_campaign_name,
        'date_sent': outlier_date,
        'subject_line': "Super Sale! Unbeatable Prices Inside!",
        'platform': random.choice(platforms),
        'country': random.choice(countries),
        'recipient_count': random.randint(100000, 1000000),  # Extremely high recipient count
        'delivered_count': random.randint(90000, 900000),    # Extremely high delivered count
        'opened_count': random.randint(50000, 500000),        # Extremely high opened count
        'clicked_count': random.randint(10000, 100000),       # Extremely high clicked count
        'bounced_count': random.randint(0, 5000),             # High but plausible
        'unsubscribe_count': random.randint(0, 1000),         # High but plausible
        'spam_report_count': random.randint(0, 500),          # High but plausible
        'conversions': random.randint(1000, 10000)            # Extremely high conversions
    }
    outlier_records.append(outlier_record)

# Convert outliers to DataFrame
outliers_df = pd.DataFrame(outlier_records)

# Concatenate outliers with the main DataFrame
email_campaigns_df = pd.concat([email_campaigns_df, outliers_df], ignore_index=True)

# Shuffle the DataFrame to mix outliers and duplicates
email_campaigns_df = email_campaigns_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Parse 'date_sent' column to datetime for sorting
email_campaigns_df['parsed_date'] = pd.to_datetime(email_campaigns_df['date_sent'], dayfirst=True, errors='coerce')

# Sort by 'parsed_date'
email_campaigns_df = email_campaigns_df.sort_values(by='parsed_date').drop('parsed_date', axis=1).reset_index(drop=True)

# Save to CSV
email_campaigns_df.to_csv('email_campaigns_dirty.csv', index=False)
print("Dirty Email Campaigns Data Generated:")
print(email_campaigns_df.head())


Dirty Email Campaigns Data Generated:
   campaign_id                   campaign_name   date_sent  \
0   campaign_2  Campaign 2 - Brother Promotion  2024-08-01   
1  campaign_18  Campaign 18 - Almost Promotion  2024-08-01   
2  campaign_43    Campaign 43 - Save Promotion  2024-08-01   
3  campaign_24    Campaign 24 - Play Promotion  2024-08-01   
4  campaign_38    Campaign 38 - True Promotion  2024-08-01   

                   subject_line      platform         country  \
0         Your Feedback Matters          GAds  United Kingdom   
1     Welcome to Our Newsletter    Google Ads           India   
2    New Arrivals Available Now        FB Ads          France   
3  Unlock Your Premium Features  LinkedIn Ads          France   
4         Your Feedback Matters  LinkedIn Ads           China   

   recipient_count  delivered_count  opened_count  clicked_count  \
0            49368            35694         21778          14918   
1            25670             7756          1289            9

### Social Media

In [8]:
# Set random seed for reproducibility
random.seed(42)
fake.seed_instance(42)

# Parameters
num_posts = 300  # Total number of social media posts
records_per_post = 1  # One record per post
total_records = num_posts * records_per_post

# Define possible platforms, content types, and content themes
platforms = ['Facebook', 'Twitter', 'Instagram', 'LinkedIn', 'Pinterest', 'TikTok']
content_types = ['Image', 'Video', 'Text', 'Carousel', 'Story', 'Live']
content_themes = ['Promotion', 'Engagement', 'Information', 'Behind-the-Scenes', 'User-Generated', 'Event', 'Product Launch']

# Define countries if needed for geo-targeting (optional)
countries = ['United States', 'Canada', 'United Kingdom', 'Australia', 'Germany', 'France', 'India', 'Brazil', 'Japan', 'China']

# Generate a list of dates for the past 3 months
date_range = pd.date_range(end=pd.Timestamp.today(), periods=90).tolist()

# Initialize list to hold all social media records
social_media_posts = []

for post_num in range(1, num_posts + 1):
    post_id = f'post_{post_num}'
    platform = random.choice(platforms)
    content_type = random.choice(content_types)
    content_theme = random.choice(content_themes)
    
    # Introduce inconsistent date formats (10% of the time)
    if random.random() > 0.1:
        date_posted = random.choice(date_range).strftime('%Y-%m-%d')
    else:
        date_posted = random.choice(date_range).strftime('%d/%m/%Y')
    
    # Introduce missing values (5% for 'content_theme' and 'sentiment_score')
    content_theme = content_theme if random.random() > 0.05 else None
    sentiment_score = round(random.uniform(-1, 1), 2) if random.random() > 0.05 else None  # -1 (negative) to 1 (positive)
    
    # Introduce erroneous entries (2% negative values for numeric metrics)
    organic_impressions = random.randint(1000, 100000) if random.random() > 0.02 else -random.randint(1000, 100000)
    paid_impressions = random.randint(0, 50000) if random.random() > 0.02 else -random.randint(0, 50000)
    organic_engagements = random.randint(100, 10000) if random.random() > 0.02 else -random.randint(100, 10000)
    paid_engagements = random.randint(0, 5000) if random.random() > 0.02 else -random.randint(0, 5000)
    clicks = random.randint(10, 2000) if random.random() > 0.02 else -random.randint(10, 2000)
    reach = random.randint(1000, 100000) if random.random() > 0.02 else -random.randint(1000, 100000)
    followers_gain = random.randint(0, 500) if random.random() > 0.02 else -random.randint(0, 500)
    conversions = random.randint(0, 300) if random.random() > 0.02 else -random.randint(0, 300)
    
    # Assign country (optional, based on platform or content)
    country = random.choice(countries) if platform in ['Facebook', 'Instagram', 'LinkedIn'] else None
    
    # Create the social media post record
    post_record = {
        'post_id': post_id,
        'platform': platform,
        'content_type': content_type,
        'content_theme': content_theme,
        'date_posted': date_posted,
        'organic_impressions': organic_impressions,
        'paid_impressions': paid_impressions,
        'organic_engagements': organic_engagements,
        'paid_engagements': paid_engagements,
        'clicks': clicks,
        'reach': reach,
        'followers_gain': followers_gain,
        'conversions': conversions,
        'sentiment_score': sentiment_score,
        'country': country
    }
    
    social_media_posts.append(post_record)

# Convert to DataFrame
social_media_df = pd.DataFrame(social_media_posts)

# Introduce Duplicates (2% of the data)
duplicates = social_media_df.sample(frac=0.02, random_state=42)
social_media_df = pd.concat([social_media_df, duplicates], ignore_index=True)

# Introduce Additional Outliers by adding some extreme values (15 records)
outlier_records = []
for _ in range(15):
    outlier_post_num = random.randint(1, num_posts)
    outlier_post_id = f'post_{outlier_post_num}'
    outlier_platform = random.choice(platforms)
    outlier_content_type = random.choice(content_types)
    outlier_content_theme = random.choice(content_themes)
    outlier_date = random.choice(date_range).strftime('%Y-%m-%d')
    
    outlier_record = {
        'post_id': outlier_post_id,
        'platform': outlier_platform,
        'content_type': outlier_content_type,
        'content_theme': outlier_content_theme,
        'date_posted': outlier_date,
        'organic_impressions': random.randint(1000000, 5000000),  # Extremely high organic impressions
        'paid_impressions': random.randint(500000, 2000000),     # Extremely high paid impressions
        'organic_engagements': random.randint(50000, 300000),    # Extremely high organic engagements
        'paid_engagements': random.randint(25000, 150000),       # Extremely high paid engagements
        'clicks': random.randint(5000, 100000),                  # Extremely high clicks
        'reach': random.randint(1000000, 5000000),               # Extremely high reach
        'followers_gain': random.randint(1000, 5000),            # Extremely high followers gain
        'conversions': random.randint(500, 3000),                # Extremely high conversions
        'sentiment_score': round(random.uniform(0.5, 1.0), 2),   # Highly positive sentiment
        'country': random.choice(countries)                      # Assign country
    }
    outlier_records.append(outlier_record)

# Convert outliers to DataFrame
outliers_df = pd.DataFrame(outlier_records)

# Concatenate outliers with the main DataFrame
social_media_df = pd.concat([social_media_df, outliers_df], ignore_index=True)

# Shuffle the DataFrame to mix outliers and duplicates
social_media_df = social_media_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Parse 'date_posted' column to datetime for sorting
social_media_df['parsed_date'] = pd.to_datetime(social_media_df['date_posted'], dayfirst=True, errors='coerce')

# Sort by 'parsed_date'
social_media_df = social_media_df.sort_values(by='parsed_date').drop('parsed_date', axis=1).reset_index(drop=True)

# Save to CSV
social_media_df.to_csv('social_media_dirty.csv', index=False)
print("Dirty Social Media Data Generated:")
print(social_media_df.head())


Dirty Social Media Data Generated:
    post_id   platform content_type   content_theme date_posted  \
0  post_198   LinkedIn         Text      Engagement  21/07/2024   
1  post_231    Twitter         Live  Product Launch  27/07/2024   
2   post_13    Twitter        Story  User-Generated  31/07/2024   
3   post_91  Instagram         Text           Event  01/08/2024   
4   post_11     TikTok        Story  Product Launch  01/08/2024   

   organic_impressions  paid_impressions  organic_engagements  \
0                91146             26794                 2635   
1                38707            -45468                 5099   
2                27772             20590                 6584   
3                 5888             11473                 2980   
4                29016              3842                  135   

   paid_engagements  clicks  reach  followers_gain  conversions  \
0              4795    1760  75012             141          203   
1               186    1525  27712   

### Advertising

In [5]:
# Set random seed for reproducibility
random.seed(42)
fake.seed_instance(42)

# Parameters
num_ads = 100
num_records_per_ad = 1000
total_days = 90  # Approximately 3 months

# Initialize list to hold all advertising records
advertising = []

# Define platform names with inconsistencies
ad_platforms = ['Google Ads', 'Facebook Ads', 'Instagram Ads', 'LinkedIn Ads', 'GAds', 'FB Ads', 'Insta Ads']

# Generate a list of dates for the past 3 months
date_range = pd.date_range(end=pd.Timestamp.today(), periods=total_days).tolist()

# Generate Advertising Metrics Data
ad_ids = [f'ad_{i+1}' for i in range(num_ads)]

for ad_id in ad_ids:
    # Assign a base date for the ad to introduce some trends
    base_date = fake.date_between(start_date='-3m', end_date='today')
    
    for _ in range(num_records_per_ad):
        # Select a date from the date range
        date = random.choice(date_range)
        
        # Optionally, introduce some trend by slightly adjusting the date
        # For example, more records on weekdays vs weekends
        if date.weekday() < 5:  # Weekday
            date_str = date.strftime('%Y-%m-%d')
        else:  # Weekend
            date_str = date.strftime('%d/%m/%Y') if random.random() < 0.1 else date.strftime('%Y-%m-%d')
        
        # Introduce missing values (5% for 'platform')
        platform = random.choice(ad_platforms) if random.random() > 0.05 else None
        
        # Introduce erroneous entries (2% negative values)
        impressions = random.randint(100, 10000) if random.random() > 0.02 else -random.randint(100, 10000)
        clicks = random.randint(0, 500) if random.random() > 0.02 else -random.randint(0, 500)
        cost_usd = round(random.uniform(50, 5000), 2) if random.random() > 0.02 else -round(random.uniform(50, 5000), 2)
        conversions = random.randint(0, 100) if random.random() > 0.02 else -random.randint(0, 100)
        
        ad_record = {
            'ad_id': ad_id,
            'platform': platform,
            'date': date_str,
            'impressions': impressions,
            'clicks': clicks,
            'cost_usd': cost_usd,
            'conversions': conversions
        }
        
        advertising.append(ad_record)

# Convert to DataFrame
advertising_df = pd.DataFrame(advertising)

# Introduce Duplicates (2% of the data)
duplicates = advertising_df.sample(frac=0.02, random_state=42)
advertising_df = pd.concat([advertising_df, duplicates], ignore_index=True)

# Introduce Additional Outliers by adding some extreme values (50 records)
outlier_records = []
for _ in range(50):
    outlier_date = random.choice(date_range).strftime('%Y-%m-%d')
    outlier_record = {
        'ad_id': random.choice(ad_ids),
        'platform': random.choice(ad_platforms),
        'date': outlier_date,
        'impressions': random.randint(10000, 1000000),  # Extremely high impressions
        'clicks': random.randint(1000, 10000),            # Extremely high clicks
        'cost_usd': round(random.uniform(10000, 100000), 2),  # Extremely high cost
        'conversions': random.randint(100, 1000)        # Extremely high conversions
    }
    outlier_records.append(outlier_record)

# Convert outliers to DataFrame
outliers_df = pd.DataFrame(outlier_records)

# Concatenate outliers with the main DataFrame
advertising_df = pd.concat([advertising_df, outliers_df], ignore_index=True)

# Shuffle the DataFrame to mix outliers and duplicates
advertising_df = advertising_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Optionally, sort by date to simulate a time series
advertising_df['parsed_date'] = pd.to_datetime(advertising_df['date'], dayfirst=True, errors='coerce')
advertising_df = advertising_df.sort_values(by='parsed_date').drop('parsed_date', axis=1).reset_index(drop=True)

# Save to CSV
advertising_df.to_csv('advertising_metrics_dirty.csv', index=False)
print("Dirty Advertising Metrics Data Generated:")
print(advertising_df.head())


/tmp/ipykernel_86239/1503101099.py:90: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  advertising_df['parsed_date'] = pd.to_datetime(advertising_df['date'], dayfirst=True, errors='coerce')


Dirty Advertising Metrics Data Generated:
    ad_id       platform        date  impressions  clicks  cost_usd  \
0  ad_100           None  2024-07-21         5998      96   2079.73   
1   ad_49  Instagram Ads  2024-07-21         4711     442   2211.19   
2   ad_63     Google Ads  2024-07-21         4887     386   3659.98   
3   ad_29      Insta Ads  2024-07-21         4754     496   2439.85   
4   ad_72   Facebook Ads  2024-07-21         8379     222   3619.68   

   conversions  
0           19  
1           16  
2          -79  
3           54  
4           96  
